The following assignment consists of a theoretical part (learning portfolio) and a practical part (assignment). The goal is to build a classification model that predicts from which subject area a certain abstract originates. The plan would be that next week we will discuss your learnings from the theory part, that means you are relatively free to fill your Learning Portfolio on this new topic and in two weeks we will discuss your solutions of the Classification Model.

#Theory part (filling your Learning Portfolio, May 10)

In preparation for the practical part, I ask you to familiarize yourself with the following resources in the next week:

1) Please watch the following video:

https://course.fast.ai/Lessons/lesson4.html

You are also welcome to watch the accompanying Kaggle notebook if you like the video.

2) In addition to the video, I recommend you to read the first chapters of the course

https://huggingface.co/learn/nlp-course/chapter1/1


Try to understand principle processes and log them in your learning portfolio! A few suggestions: What is a pre-trained NLP model? How do I load them? What is tokenization? What does fine-tuning mean? What types of NLP Models are there? What possibilities do I have with the Transformers package? etc...

#Practical part (Assignment, May 17)

1) Preprocessing: The data which I provide as zip in Olat must be processed first, that means we need a table which has the following form:

Keywords | Title | Abstract | Research Field

The research field is determined by the name of the file.

2) We need a training dataset and a test dataset. My suggestion would be that for each research field we use the first 5700 lines for the training dataset and the last 300 lines for the test dataset. Please stick to this because then we can compare our models better!

3) Please use a pre-trained model from huggingface to build a classification model that tries to predict the correct research field from the 26. Please calculate the accuracy and the overall accuracy for all research fields. If you solve this task in a group, you can also try different pre-trained models. In addition to the abstracts, you can also see if the model improves if you include keywords and titles.

Some links, which can help you:

https://huggingface.co/docs/transformers/training

https://huggingface.co/docs/transformers/tasks/sequence_classification

One last request: Please always use PyTorch and not TensorFlow!

In [4]:
!pip install transformers==4.28.0 accelerate datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import pandas as pd
import glob 
import os 
from google.colab import drive
from google.colab import data_table
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings

data_table.enable_dataframe_formatter()

drive.mount('/content/drive')

dir = '/content/drive/MyDrive/Colab Notebooks/data 2/'

'''had to skip bad lines due to errors when reading in'''

train_data = pd.DataFrame()
test_data = pd.DataFrame()

for filename in os.listdir(dir):
    if filename.endswith('.csv'): 
        file_path = os.path.join(dir, filename)
        data = pd.read_csv(file_path, on_bad_lines="skip")

        research_field = filename.split('_')[0]   
        data['Research Field'] = research_field
        data['Keywords'] = data['Author Keywords'].fillna('') + ' ' + data['Index Keywords'].fillna('')

        columns_to_keep = ['Keywords', 'Research Field', 'Abstract', 'Title']
        data = data[columns_to_keep]

        # Replace empty abstracts with title and keywords
        data.loc[data['Abstract'] == '[No abstract available]', 'Abstract'] = data['Title'] + ' ' + data['Keywords']

        # Split into Training and Test
        train_end_idx = int(len(data) * 0.95)
        train_data = pd.concat([train_data, data[:train_end_idx]])
        test_data = pd.concat([test_data, data[train_end_idx:]])

# Split into training and validation using stratified sampling
train_data, validation_data = train_test_split(train_data, test_size=0.20, stratify=train_data['Research Field'], random_state=42)

print("Length of training, validation and test set")
print((len(train_data), len(validation_data), len(test_data)))

validation_data.head(10)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-6-7eded418c50c>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[data['Abstract'] == '[No abstract available]', 'Abstract'] = data['Title'] + ' ' + data['Keywords']
<ipython-input-6-7eded418c50c>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[data['Abstract'] == '[No abstract available]', 'Abstract'] = data['Title'] + ' ' + data['Keywords']
<ipython-input-6-7eded418c50c>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

Length of training, validation and test set
(117490, 29373, 7730)


,Keywords,Research Field,Abstract,Title
1197,article; body height; body mass; body weight;...,MEDI,AIM: To describe the methods used to construct...,WHO Child Growth Standards based on length/hei...
61,Computer science; Evolutionary algorithms; Ne...,MATH,The study manipulated the neighborhood topolog...,Small worlds and mega-minds: Effects of neighb...
1809,Controlling sparsity; Logical OLAP design; Nor...,DECI,A poorly designed OLAP (on-line analytical pro...,Normalising OLAP cubes for controlling sparsity
1260,,CHEM,Sol-gel routes to metal oxide nanoparticles in...,Nonaqueous sol-gel routes to metal oxide nanop...
455,adult; article; case report; cephalometry; ch...,DENT,The smile arc is defined as the relationship o...,The importance of incisor positioning in the e...
1316,Archaean; crustal evolution; geochronology; I...,EART,The Itsaq Gneiss Complex of southern West Gree...,The Itsaq Gneiss Complex of southern West Gree...
1017,aged; aggression; article; dementia; educatio...,NURS,Caring for cognitively impaired aggressive res...,Aggressive behavior. Educating nursing assista...
1821,alpha tocopherol; alpha tocopheryl polyethyle...,PHAR,Berberine chloride (BBR) is a natural isoquino...,Bioavailability study of berberine and the enh...
1275,Caring science; Rogers' science of unitary hum...,NURS,Background. Two dominant discourses in contemp...,Caring science and the science of unitary huma...
415,Animalia; Canis familiaris; biological marker...,VETE,Objective - To determine whether serum concent...,Serum cardiac troponin I and cardiac troponin ...


In [7]:
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, AutoModelForSequenceClassification, Trainer
from datasets import Dataset,DatasetDict

# Make data comply to transformers standard: text and labels
train_data_transformers = train_data.loc[:, ['Abstract', 'Research Field']]
train_data_transformers.columns = ['text', 'labels']
test_data_transformers = test_data.loc[:, ['Abstract', 'Research Field']]
test_data_transformers.columns = ['text', 'labels']
valid_data_transformers = validation_data.loc[:, ['Abstract', 'Research Field']]
valid_data_transformers.columns = ['text', 'labels']

# convert to datasets
train_ds = Dataset.from_pandas(train_data_transformers)
valid_ds = Dataset.from_pandas(valid_data_transformers)
test_ds = Dataset.from_pandas(test_data_transformers)

dataset_dict = DatasetDict({'train': train_ds, 'validation': valid_ds, 'test': test_ds})

# model and tokenizer definition
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

id2label = {1: 'DENT', 2: 'AGRI', 3: 'ENER', 4: 'PSYC', 5: 'DECI', 6: 'VETE', 7: 'PHAR', 8: 'MATH',
       9: 'NURS', 10: 'ECON', 11: 'COMP', 12: 'ARTS', 13: 'CENG', 14: 'ENVI', 15: 'SOCI', 16: 'BIOC',
       17: 'MATE', 18: 'CHEM', 19: 'HEAL', 20: 'ENGI', 21: 'BUSI', 22: 'NEUR', 23: 'MEDI', 24: 'IMMU',
       25: 'PHYS', 0: 'EART'}
label2id = {value: key for key, value in id2label.items()}

# tokenize text and convert labels into labekl id
def tokenize_function(x):
    tokens = tokenizer(x['text'], truncation=True, padding="max_length")
    tokens["labels"] = [label2id[label] for label in x["labels"]]
    return tokens

tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)
     

Map:   0%|          | 0/117490 [00:00<?, ? examples/s]

Map:   0%|          | 0/29373 [00:00<?, ? examples/s]

Map:   0%|          | 0/7730 [00:00<?, ? examples/s]

In [ ]:
from transformers import BertForSequenceClassification, TrainingArguments, DataCollatorWithPadding, Trainer
import evaluate
import numpy as np

acc = evaluate.load("acc")

def eval_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return acc.compute(predictions=predictions, references=labels)

arguments = TrainingArguments(
    "/content/drive/MyDrive/Colab Notebooks/data 2/checkpoints/",
    evaluation_strategy="steps",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_strategy="steps",
    save_steps=1000
)

def initialize_model():
    model = BertForSequenceClassification.from_pretrained(checkpoint, num_labels=26, id2label=id2label, label2id=label2id)
    return model

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model_init=initialize_model,
    args=arguments,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    eval_metrics=eval_metrics
)

trainer.train()#resume_from_checkpoint=True 

preds = trainer.predict(tokenized_datasets["test"])
preds


'''have a problem with the checkpoint function. 
unfortunately the training would take hours and crash so 
I couldn't write the code further from this point on '''

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss,Validation Loss


In [5]:
from collections import defaultdict, Counter

correct_single_hits = 0
corect_highest_three_categories = 0
incorrect_single_hits_per_category = defaultdict(int)
incorrect_triple_hits_per_category = defaultdict(int)

for pred, example in zip(preds.predictions, tokenized_datasets["test"]):
    predicted_label = np.argmax(pred)
    true_label = example["labels"]
  
    if predicted_label == true_label:
        correct_single_hits += 1

    top_three_categories = np.argpartition(pred, -3)[-3:]
    if true_label in top_three_categories:
        corect_highest_three_categories += 1
    else:
        incorrect_single_hits_per_category[id2label[true_label]] += 1
        incorrect_triple_hits_per_category[id2label[true_label]] += 1

sorted_single_hit_fails = dict(Counter(incorrect_single_hits_per_category).most_common())
sorted_triple_hit_fails = dict(Counter(incorrect_triple_hits_per_category).most_common())

accuracy_single_hit = correct_single_hits / len(preds.predictions)
accuracy_triple_hit = corect_highest_three_categories / len(preds.predictions)

print("Single and Triple Hit Accuracy")
print((accuracy_single_hit, accuracy_triple_hit))
print("Category to wrong prediction single")
print(sorted_single_hit_fails)
print("Category to wrong prediction triple")
print(sorted_triple_hit_fails)



╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 8>:8                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'predictions' is not defined

Addition: Accuracy measures whether the research field with the highest probability value matches the target. With 26 research fields, it would also be interesting to know if the correct target is at least among the three highest probability values.

$\begin{pmatrix} A\\ B \\ C \\D \\E \end{pmatrix} = \begin{pmatrix} 0.1\\ 0.95 \\ 0.5 \\0.2 \\0.3 \end{pmatrix} → \text{Choice}_1 = B, \text{Choice}_3 = B,C,E$